In [19]:
import os
from dotenv import load_dotenv
import boto3
import redshift_connector

In [20]:
load_dotenv()

client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
)

In [21]:
conn = redshift_connector.connect(
    host=os.getenv('REDSHIFT_HOST'),
    database=os.getenv('REDSHIFT_DATABASE'),
    user=os.getenv('REDSHIFT_USER'),
    password=os.getenv('REDSHIFT_PASSWORD'),
    port=os.getenv('REDSHIFT_PORT')
)

In [22]:
cursor = conn.cursor()

In [23]:
def load_file_to_redshift(file_name):
    table_name = file_name.split('.')[0]
    client.upload_file(
        Filename='./data/stage/{}'.format(file_name),
        Bucket=os.getenv('AWS_BUCKET_NAME'),
        Key='course_etl_target/{}'.format(file_name),
    )
    print('Archivo {} cargado'.format(file_name))
    sentence = '''
    copy public.{} from 's3://{}/course_etl_target/{}'
    credentials 'aws_iam_role={}' csv delimiter '|' region 'us-east-2' ignoreheader 1;
    '''.format(table_name, os.getenv('AWS_BUCKET_NAME'), file_name, os.getenv('AWS_ROLE_ARN'))
    try:
        cursor.execute(sentence)
        print('Tabla {} cargada'.format(table_name))
    except Exception as e:
        print('Error en la tabla {}: {}'.format(table_name, e))

In [27]:
load_file_to_redshift('countries.csv')
load_file_to_redshift('codes.csv')
load_file_to_redshift('quantity.csv')
load_file_to_redshift('flow.csv')
load_file_to_redshift('years.csv')
load_file_to_redshift('trades.csv')

Archivo countries.csv cargado
Error en la tabla countries: connection is closed


In [25]:
conn.commit()

In [26]:
conn.close()